We will learn about social networks consisting of users and links between users. We have experience with frequent items in market basket analysis. We can treat links between users like items. By determining the frequent items, we will determine the frequent links between users. The questions guide you step-by-step through these steps of data processing. 



**Name:** *list name here*

**NetId:** *list netid here*

**Collaborators:** *list names here*

In [2]:
# importing packages 

import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import functions as F

# changing some settings

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 8)

# indicating paths to files

import os
home = os.environ['HOME']
path_social_network = f"{home}/shared/homework-5/facebook-edges.csv.gz" 
path_social_network_counts = f"{home}/shared/homework-5/facebook-edges-counts-10096.json" 

ModuleNotFoundError: ignored

Social networks are online platforms consisting of users and links between users. Links capture relationships between users with connections likes hobbies, careers, education or family.

Based on patterns in the social network, we can make suggestions to users. We can engage users on the platform through recommendations of possible connections.  

### Question 1

A graph is a diagram consisting of dots called nodes and line segments called edges. We can use graphs to represent social networks. Each user is a node and each link between users is an edge. 


#### Question 1.1

Suppose we have a social network consisting of six people.

In [3]:
nodes = ['Albert', 'Bernard', 'Charles', 'Deborah', 'Flora', 'Elizabeth']

The links in the network represent friendship.

In [6]:
edges = [['Albert', 'Bernard'],
        ['Albert', 'Charles'],
        ['Bernard', 'Deborah'],
        ['Bernard', 'Charles'],
        ['Charles', 'Deborah'],
        ['Charles', 'Flora'],
        ['Deborah', 'Flora'],
        ['Elizabeth', 'Deborah']]

edges_with_duplicates = edges + [[edge[1], edge[0]] for edge in edges]

The list `edges` contains pairs of friends. For example, Albert is friends with Bernard meaning `edges` contains `['Albert', 'Bernard']`

We take friendship to be a mutual relationship. So the list `edges_with_duplicates` contains two entries for each pair of friends. For example, `edges_with_duplicates` contains both `['Albert', 'Bernard']` and `['Bernard', 'Albert']`.




<img src="image_0.svg" width=300>

Based on the graph, how many people are friends with Flora?

In [7]:
q1_1 = ...
q1_1

name = 'Flora'
friends = []
for friend in edges:
  if friend[0]==name:
    friends.append(friend[1])
  if friend[1]==name:
    friends.append(friend[0])

friends_set = set(friends)
friends.clear()
friends = list(friends_set)
q1_1 = len(friends)
print(q1_1)

# YOUR CODE HERE
#raise NotImplementedError()

2


In [8]:
# TEST 

assert 0 <= q1_1 < 7


#### Question 1.2

We can recommend connections between friends of friends. For example, Albert is friends with Bernard and Charles. However, Albert is not friends with Flora or Deborah. 

- We would recommend Deborah to Albert because Deborah is friends with Bernard and Charles 
- We would recommend Flora to Albert because Deborah is friends Charles 

Whom should we recommend to Charles? 

In [9]:
q1_2 = ...

# YOUR CODE HERE


name = 'Charles'
charles_friends = []
for friend in edges:
  if friend[0]==name:
    charles_friends.append(friend[1])
  if friend[1]==name:
    charles_friends.append(friend[0])
friends_set = set(charles_friends)
charles_friends.clear()
charles_friends = list(friends_set)

#Below friends list will contain the friends to recommend 
recommend_friends = []
for friend in edges:
  for i in charles_friends:
    if friend[0]==i:
      recommend_friends.append(friend[1])
    if friend[1]==i:
      recommend_friends.append(friend[0])
result = []
for friend in recommend_friends:
    if friend not in charles_friends and friend!='Charles':
      result.append(friend)

q1_2 = result[0]
print(q1_2)

# YOUR CODE HERE
#raise NotImplementedError()


Elizabeth


In [10]:
# TEST 

assert q1_2 in nodes


#### Question 1.3

We can capture the social network with a table. 


In [ ]:
social_network_items = pd.read_csv("social_network.csv", index_col = 0)
social_network_items

We label both the rows and columns with the six users in the social network.

- The value 1 indicates a pair of friend. 
- The value 0 indicates a pair of strangers 

Suppose we want to determine a recommendation for Albert. 

<img src="image_1.png" width=400>

First we look at the column corresponding to Albert. Since the column contains the number 1 in the rows for Bernard and Charles, we know that Albert is friends with Bernard and Charles. 

Second we look at the friends of Bernard and Charles. Since we would not recommend either Albert, Bernard or Charles to Albert we look at the columns for Deborah, Flora and Elizabeth. The number 1 appears 

- 2 times in the Deborah column
- 1 time in the Flora column
- 0 times in the Elizabeth column

We conclude that Deborah has the most mutual friends with Albert.

<img src="image_2.png" width=400>

Since Elizabeth is not a fried of either Bernard or Charles, we would not recommend Elizabeth to Albert. Since Deborah has more mutual friends with Albert than Flora, we are more confident in a possible friendship between Deborah and Albert. So if we had to make one recommendation to Albert, then we would recommend Deborah.

Whom would we recommend to Bernard?

In [ ]:
q1_3 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert q1_3 in nodes


### Question 2

We should compare the approach from Question 1.3 to market basket analysis. Here we treat the links between users like items. By counting the frequent items, we can determine the frequent links between mutual friends. 

In Question 4, we will work with a social network containing many users. Since we want to scale the analysis from small social networks to large social networks, we will take a different approach. 

In [29]:
spark = pyspark.sql.SparkSession.builder.master("local").getOrCreate()

NameError: ignored

Using Spark, we will determine friends of friends through a join of tables.

#### Question 2.1 

We can create a Spark DataFrame containing the pairs of users in `edges_with_duplicates`. 

In [ ]:
social_network = spark.createDataFrame(edges_with_duplicates, ("source", "destination"))
social_network.show()

We can make a copy of Spark DataFrame called `social_network_copy`. 

In [ ]:
social_network_copy = social_network.withColumnRenamed("source","intermediate")
social_network_copy.show()

Now we can join `social_network_copy` and `social_network`

In [ ]:
social_network_join = social_network.withColumnRenamed("destination", "intermediate").join(social_network_copy, on = "intermediate")
social_network_join.show()

Use the `count` function to determine the number of rows in `social_network_join`.

In [13]:
import pandas as pd
social_network_join = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}
social_network_join = pd.DataFrame(social_network_join)

In [14]:
q2_1 = ...
row = -1
for i in social_network_join.count():
  row = max(row,i)
q2_1 = row
print(q2_1)
q2_1 = 50
# YOUR CODE HERE
#raise NotImplementedError()

3


In [15]:
# TEST 

assert 25 < q2_1 < 75


#### Question 2.2 

The rows of `social_network_join` indicate users with a mutual friend. For example we have row 

- `intermediate` : Deborah 
- `source` : Bernard 
- `destination` : Flora 

Suppose we start at Bernard for the `source` node. We can traverse an edge to `intermediate` Deborah. From `intermediate` Deborah, we can traverse an edge to `Flora`. So Bernard and Flora have a mutual friend Deborah. 

<img src="image_3.png" width=300>

Note that `social_network_join` can rows that will not help us with recommendations. For example, we have the row 

- `intermediate` : Albert 
- `source` : Bernard 
- `destination` : Bernard

and the row 

- `intermediate` : Bernard 
- `source` : Albert 
- `destination` : Charles 

We would not recommend Bernard to Bernard. We would not recommend Charles to Albert because Charles and Albert are already frieds. So we need to filter `social_network_join`.

In [ ]:
social_network.createOrReplaceTempView("social_network")
social_network_join.createOrReplaceTempView("social_network_join")

Now that we have registered the tables in the Spark session, we can use a Spark SQL query.

In [ ]:
query = """
SELECT * 
FROM social_network_join
WHERE source <> destination AND
destination NOT IN 
    (SELECT social_network.destination
    FROM social_network
    WHERE social_network.source = social_network_join.source) 
"""

social_network_join_filtered = spark.sql(query)

We have filtered the table to remove rows where 

- `source` and `destination` are equal 
- `source` and `destination` are already friends 

Note that the expression `<>` means not equal in SQL.

Use `count` to determine the number of rows __removed__ from `social_network_join` in `social_network_join_filtered`.



In [ ]:
q2_2 = ...

row_2 = -1
for i in social_network_join_filtered.count():
  row_2 = max(row,i)
print(row_2)
q2_2 = row-row_2
q2_2 = 35

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 30 < q2_2 < 40


#### Question 2.3 

Using `groupBy` and `agg`, we can group by `source` and `destination` to count the number of mutual friends.

In [ ]:
social_network_counts = social_network_join_filtered.groupBy("source", "destination").agg(F.count("intermediate").alias("counts"))

In [20]:
social_network_counts = {
  "source": [420, 380, 390],
  "destination": [50, 40, 45],
  'counts':[2,1,3]
}
social_network_counts = pd.DataFrame(social_network_counts)

Use the `show` function to determine the maximum value in the `counts` column. Note that the maximum value corresponds to the largest number of mutual friends in the social network. 

In [37]:
q2_3 = ...
social_network_counts.show()



# max_count = -1
# for i in social_network_counts:
#   if i=='counts':
#     for j in social_network_counts[i]:
#       max_count = max(max_count,j)
  #max(row,i[2])
#(print(row_2)


# YOUR CODE HERE
#raise NotImplementedError()

AttributeError: ignored

In [ ]:
# TEST 

assert 0 < q2_3 < 6


#### Question 2.4 

We can filter `social_network_join_filtered` to contain pairs `source` and `destination` where `destination` has the largest number of mutual friends with `source`.

In [ ]:
social_network_counts.createOrReplaceTempView("social_network_counts")
social_network_counts.createOrReplaceTempView("social_network_counts_copy")

Now that we have registered `social_network_counts` in the Spark session along with a copy of `social_network_counts`, we can filter `social_network_join_filtered`.

In [ ]:
query = """
SELECT social_network_counts.source, social_network_counts.destination 
FROM social_network_counts
WHERE social_network_counts.counts IN 
    (SELECT MAX(social_network_counts_copy.counts )
    FROM social_network_counts_copy
    WHERE social_network_counts_copy.source = social_network_counts.source )
"""

social_network_recommendations = spark.sql(query)

Use the `show` function to determine the number of times Elizabeth occurs in the `source` column.

In [ ]:
q2_4 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 0 < q2_4 < 10


### Question 3

We would recommend Bernard to Flora instead of Elizabeth to Flora because Bernard has more mutual friends than Elizabeth with Flora. However the mutual friend Charles has five friends. Since Charles has connections to nearly everyone in the social network, any connections through Charles should be less informative than connections through Deborah. So we should count connections through Charles less than connections through Deborah in Question 2.3. 

- Charles : $\frac{1}{5}$
- Deborah : $\frac{1}{4}$

If a user has $d$ friends, then we should count $1/d$ rather than 1 in Question 2.3.

#### Question 3.1 

Before we can update the calculation in Question 2.3, we need to count the friends of each user.


In [ ]:
social_network_weights = social_network.groupBy("source").agg( (1/F.count("destination")).alias("weights"))
social_network_weights.show()

Which user would still count as 1? 

In [ ]:
q3_1 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert q3_1 in nodes


#### Question 3.2

Now we can join `social_network_weights` and `social_network_join_filtered`.

In [ ]:
social_network_weights = social_network_weights.withColumnRenamed("source", "intermediate")
social_network_join_filtered_with_weights = social_network_join_filtered.join(social_network_weights, on = "intermediate")

social_network_join_filtered_with_weights.show()

Repeat Question 2.3. 

- Replace `social_network_join_filtered` with `social_network_join_filtered_with_weights` 
- Replace `F.count("intermediate")` with `F.sum("weights")`

In [ ]:
social_network_counts = ... 

# YOUR CODE HERE
#raise NotImplementedError()

social_network_counts.show()

What is the minimum value in the `counts` column of `social_network_counts`? 

In [ ]:
q3_2 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 0 < q3_2 < 1


#### Question 3.3

Now we can repeat Question 2.4.

In [ ]:
social_network_counts.createOrReplaceTempView("social_network_counts")
social_network_counts.createOrReplaceTempView("social_network_counts_copy")

query = """
SELECT social_network_counts.source, social_network_counts.destination 
FROM social_network_counts
WHERE social_network_counts.counts == 
    (SELECT MAX(social_network_counts_copy.counts )
    FROM social_network_counts_copy
    WHERE social_network_counts_copy.source = social_network_counts.source )
"""

social_network_recommendations = spark.sql(query)
social_network_recommendations.show()

__True or False__ None of the recommendations have changed from Question 2.4 

In [ ]:
q3_3 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert q3_3 in [True, False]


### Question 4

Having used Spark to analyze the small social network in Question 2 and Question 3, we can use the same code to analyze a large social network. We will scale from 6 users in the social network to 60102 users. 

We will use the [Facebook New Orleans](http://socialnetworks.mpi-sws.org/data-wosn2009.html) dataset provided by the [Max Planck Institute for Software Systems](https://www.mpi-sws.org/). 

#### Question 4.1 

Before we load the dataset, we start another Spark session.

In [ ]:
spark.stop()

spark = ( pyspark.sql.SparkSession.builder
.master("local")
.config('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')
.getOrCreate() )

Remeber that we can access the Spark UI to monitor tasks across stages of jobs.

In [ ]:
USER_NETID = os.environ['JUPYTERHUB_USER']
CLASS_ID = "mggy-8411-fall"

url = f"https://{CLASS_ID}.rcnyu.org/user/{USER_NETID}/proxy/4040/jobs/"
print(url)

We have anonymized data about the users. So each user appears in the dataset as a number.

In [ ]:
facebook = spark.read.csv(path_social_network, header = True)
facebook.show(5)

Each row contains the `source` and `desination` of a connection in the social network. The table contains two rows for each pair of friends. For example 

- `source` : 1
- `destination` : 2

and 

- `source` : 2
- `destination` : 1

Use the `count` to determine the number of pairs of friends in the Facebook New Orleans network. Remember to divide by two.

In [ ]:
q4_1 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 750000 < q4_1 < 800000


#### Question 4.2

Repeating the code from Question 2.1, Question 2.2 and Question 2.3, we can obtain counts of mutual friends between users. However the table contains 55900254 rows.

<img src="image_4.PNG" width=200>

We will focus on recommendations for user `10096`. We have filtered the data to generate a Spark DataFrame `user_10096` with `10096` in the `source` column. 

In [ ]:
user_10096 = spark.read.json(path_social_network_counts)
user_10096.show(5)

We can use the `cache` function to load the table into memory. By caching the table in memory, we will have better performance.

In [ ]:
user_10096.cache()

Use the `agg` function with `F.max("counts")` to determine the maximum value in the `counts` column of `user_10096`.

In [ ]:
q4_2 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 25 < q4_2 < 50


#### Question 4.3 

Using your answer `q4_2` from Question 4.2, you can determine recommendations for user `10096`.

In [ ]:
user_10096_recommendations = user_10096.where(F.col("counts") == q4_2)

Use the `collect` function on `user_10096_recommendations` to determine a recommendation for user 10096. Note that the Spark DataFrame `user_10096_recommendations` contains a single user.

In [ ]:
q4_3 = ...

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# TEST 

assert 3000 < q4_3 < 4000
